# How to evaluate llm with text descriptors?

In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import time
from datetime import timedelta

from sklearn import datasets, ensemble, model_selection

In [ ]:
from evidently.ui.workspace.cloud import CloudWorkspace

from evidently import ColumnMapping
from evidently.report import Report

from evidently.metrics import ColumnSummaryMetric, ColumnDistributionMetric, ColumnDriftMetric, DataDriftTable, TextDescriptorsDistribution

from evidently.metric_preset import DataDriftPreset, DataQualityPreset, TextOverviewPreset, TextEvals

from evidently.descriptors import HuggingFaceModel, OpenAIPrompting 
from evidently.descriptors import RegExp, BeginsWith, EndsWith, Contains, DoesNotContain, IncludesWords, ExcludesWords
from evidently.descriptors import TextLength, OOV, NonLetterCharacterPercentage, SentenceCount, WordCount, Sentiment

In [ ]:
import nltk
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

# Load Data

In [ ]:
assistant_logs =  pd.read_csv('chat_df.csv', index_col=0, parse_dates=['start_time', 'end_time'])
assistant_logs.index = assistant_logs.start_time
assistant_logs.index.rename('index', inplace=True)

In [ ]:
assistant_logs[["question", "response"]].head()

In [ ]:
assistant_logs.iloc[6].question

In [ ]:
assistant_logs.iloc[6].response

# One-off reports

In [ ]:
column_mapping = ColumnMapping(
    datetime='start_time',
    datetime_features=['end_time'],
    text_features=['question', 'response'],
    categorical_features=['organization', 'model_ID', 'region', 'environment', 'feedback'],
)

### Simple descriptors

In [ ]:
#Built-in descriptors without parameters
report = Report(metrics=[
    TextEvals(column_name="question", descriptors=[
        Sentiment(display_name="Question sentiment"),
        TextLength(display_name= "Question length"),
        OOV(display_name= "Question out of vocabulary words")
    ]),
    TextEvals(column_name="response", descriptors=[
        Sentiment(display_name="Response sentiment"),
        NonLetterCharacterPercentage(display_name="Non letter characters in response"),
        SentenceCount(display_name="Sentence count in response"),
        WordCount(display_name="Word count in response")
    ])
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)], 
           column_mapping=column_mapping)
report    

In [ ]:
#Built-in descriptors with parameters
report = Report(metrics=[
    TextEvals(column_name="question", descriptors=[
        BeginsWith(display_name="'How' question", prefix="How"),
        RegExp(reg_exp=r"^I", display_name= "Question begins with 'I'"),
        IncludesWords(words_list=['invoice', 'salary'], display_name="Questions about invoices and salary")
    ]),
    TextEvals(column_name="response", descriptors=[
        EndsWith(display_name="Assisrance might be needed", suffix="for assistance."),
        ExcludesWords(words_list=['wrong', 'mistake'], display_name="Responses without mention of mistakes"),
        Contains(items=['medical leave'], display_name="contains 'medical leave'"),
        DoesNotContain(items=['employee portal'], display_name="does not contain 'employee portal'")
    ])
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)], 
           column_mapping=column_mapping)
report    

### LLM-based descriptors

In [ ]:
pii_prompt = """
Personally identifiable information (PII) is information that, when used alone or with other relevant data, can identify an individual.

PII may contain direct identifiers (e.g., passport information) that can identify a person uniquely, 
or quasi-identifiers (e.g., race) that can be combined with other quasi-identifiers (e.g., date of birth) to successfully recognize an individual.
PII may contain person's name, person's address,and something I may forget to mention

Please identify whether or not the above text contains PII

text: REPLACE 

Use the following categories for PII identification:
1 if text contains PII
0 if text does not contain PII
0 if the information provided is not sufficient to make a clear determination

Retrun a category only
"""

negativity_prompt = """
Classify text into two groups: negative and positive

text: REPLACE 

Use the following categories for classification:
NEGATIVE if text is negative
POSITIVE if text is NOT negative
UNKNOWN use this category only if the information provided is not sufficient to make a clear determination

Retrun only category
"""

In [ ]:
#Descriptors with external models
#to run OpenAIPrompting descriptor make sure you set environement variable with openai token 
report = Report(metrics=[
    TextEvals(column_name="response", descriptors=[
        HuggingFaceModel("toxicity", "DaNLP/da-electra-hatespeech-detection", {"module_type": "measurement"}, {"toxic_label": "offensive"}, "toxicity", display_name="Hugging Face Toxicity for response"),
        OpenAIPrompting(prompt=pii_prompt, prompt_replace_string="REPLACE", model="gpt-3.5-turbo-instruct", feature_type="num", display_name="PII for response (by gpt3.5)"),
        OpenAIPrompting(prompt=negativity_prompt, prompt_replace_string="REPLACE", model="gpt-3.5-turbo-instruct", feature_type="cat", display_name="Negativity for response (by gpt3.5)")       
    ])
])

report.run(reference_data=assistant_logs[datetime(2024, 4, 8) : datetime(2024, 4, 9)], 
           current_data=assistant_logs[datetime(2024, 4, 9) : datetime(2024, 4, 10)], 
           column_mapping=column_mapping)

report    

## Get dataset with calculated descriptors

In [ ]:
#reference dataset enriched with descriptors
report.datasets()[0]

In [ ]:
#current dataset enriched with descriptors
report.datasets()[1]